# Broadcasting

### NumPy Broadcasting Rules

Broadcasting lets arrays of different shapes participate in arithmetic  
*without explicit looping or copying*. NumPy automatically expands dimensions  
of size `1` (or missing dimensions) to match the other operand.

#### 1. Basic Rules

1. **Align shapes from the right.**  
   Compare dimensions starting from the last one.

2. **Dimensions are compatible if:**
   - They are equal, or  
   - One of them is `1`.

3. **If compatible,** NumPy conceptually *stretches* the smaller array  
   along dimensions where it has size `1`.

If any dimension pair is incompatible, a `ValueError` is raised.

#### 2. Examples

| A shape | B shape | Result shape | Explanation |
|----------|----------|--------------|--------------|
| `(5, 3)` | `(3,)` | `(5, 3)` | Last dim matches → expand B along rows |
| `(5, 3)` | `(1, 3)` | `(5, 3)` | Leading dim 1 → repeat over 5 rows |
| `(10, 1)` | `(3,)` | `(10, 3)` | Last dim 1 → repeat each element over 3 cols |
| `(4, 3, 2)` | `(3, 1)` | `(4, 3, 2)` | Align right → middle dim matches, 1 expands |
| `(5, 3)` | `(5, 2)` | ❌ | 3 ≠ 2 and neither is 1 |

#### 3. Tips

- Use `np.newaxis` or `None` to add length-1 dimensions manually:  
  ```python
  x[:, None]     # adds new axis → (n,) → (n,1)
  x[None, :]     # adds new axis → (n,) → (1,n)
  ```
- `keepdims=True` in reduction functions keeps a dimension of size 1,  
  making broadcasting back easy.

✅ **Goal:** reshape minimally, let NumPy do the looping in C.


In [12]:
import numpy as np
rng = np.random.default_rng(0)

n_samples, n_features = 10000, 64

X = rng.standard_normal((n_samples, n_features))

## Example 0: Sum of all elements in an array

In [41]:
x1 = np.array([1, 2, 3])
x2 = np.array([[1, 2, 3]])
x3 = np.array([[1], [2], [3]])

print("🔴 x1:\n", x1)
print("   x1.shape:", x1.shape)
print("\n")
print("🔴 x2:\n", x2)
print("x2.shape:", x2.shape)
print("\n")
print("🔴 x3:\n", x3)
print("x3.shape:", x3.shape)

print("\n")

print("🟥 x1 + x2:\n", x1 + x2)
print("x1 + x2.shape:", (x1 + x2).shape)
print("\n")
print("🟥 x1 + x3:\n", x1 + x3)
print("x1 + x3.shape:", (x1 + x3).shape)
print("\n")
print("🟥 x2 + x3:\n", x2 + x3)
print("x2 + x3.shape:", (x2 + x3).shape)


🔴 x1:
 [1 2 3]
   x1.shape: (3,)


🔴 x2:
 [[1 2 3]]
x2.shape: (1, 3)


🔴 x3:
 [[1]
 [2]
 [3]]
x3.shape: (3, 1)


🟥 x1 + x2:
 [[2 4 6]]
x1 + x2.shape: (1, 3)


🟥 x1 + x3:
 [[2 3 4]
 [3 4 5]
 [4 5 6]]
x1 + x3.shape: (3, 3)


🟥 x2 + x3:
 [[2 3 4]
 [3 4 5]
 [4 5 6]]
x2 + x3.shape: (3, 3)


## Example 1: center columns of a matrix

### WRONG: explicit Python loops over columns

In [30]:
def center_columns_loop(X):
    Xc = X.copy()
    for j in range(X.shape[1]):
        col_mean = X[:, j].mean()
        Xc[:, j] = Xc[:, j] - col_mean
    return Xc

Xc_loop = center_columns_loop(X)

### RIGHT: broadcasting with keepdims

In [42]:
# Step 1: compute column means with axis=0; shape is (n_features,)
col_means = X.mean(axis=0)                    # shape: (Μ,)

# Step 2: make it align with X's 2D shape using keepdims or None
col_means_2d = X.mean(axis=0, keepdims=True)  # shape: (1, Μ)

# Either of the following broadcasts over rows:
Xc_bcast_a = X - col_means                    # (Ν,Μ) - (Μ,) -> (Ν,Μ)
Xc_bcast_b = X - col_means_2d                 # (Ν,Μ) - (1,Μ) -> (Ν,Μ)

### Compare results

In [34]:
# --- Check correctness ---------------------------------------------------------
print("Same as loop (using (64,) mean)?   ", np.allclose(Xc_bcast_a, Xc_loop))
print("Same as loop (using (1,64) mean)?  ", np.allclose(Xc_bcast_b, Xc_loop))

# --- Mini shape demo to visualize the rule ------------------------------------
print("\nX shape:", X.shape)                          # (Ν, Μ)
print("col_means shape:", col_means.shape)          # (Μ,)
print("col_means_2d shape:", col_means_2d.shape)    # (1, Μ)

Same as loop (using (64,) mean)?    True
Same as loop (using (1,64) mean)?   True

X shape: (10000, 64)
col_means shape: (64,)
col_means_2d shape: (1, 64)


## Example 2: row-wise normalization

### WRONG: explicit Python loops over rows

In [35]:
def normalize_rows_loop(X):
    Xn = X.copy()
    for i in range(X.shape[0]):                     # loop over rows
        norm = np.sqrt((Xn[i] ** 2).sum())          # compute L2 norm
        Xn[i] = Xn[i] / norm if norm != 0 else Xn[i]
    return Xn

Xn_loop = normalize_rows_loop(X)

### RIGHT: broadcasting with keepdims

In [ ]:
# Compute all row norms at once (axis=1)
row_norms = np.linalg.norm(X, axis=1, keepdims=True)   # shape (n, 1)

# Broadcasting: (n,m) / (n,1) -> (n,m)
Xn_bcast = X / row_norms

### Compare results

In [37]:
print("Equal results? ", np.allclose(Xn_bcast, Xn_loop))
print("\nX shape:", X.shape)
print("row_norms shape:", row_norms.shape)

Equal results?  True

X shape: (10000, 64)
row_norms shape: (10000, 1)


# Example 3: All-pairs Euclidean distances between two sets of vectors

Given X (shape (n, d)) and Y (shape (m, d)), compute the matrix D (shape (n, m))
with $D_{i,j} = \| X_{i} - Y_{j} \|_2$.


In [ ]:
rng = np.random.default_rng(0)

n, m, d = 4000, 3000, 64
X = rng.standard_normal((n, d))
Y = rng.standard_normal((m, d))

### WRONG: explicit Python loops over rows and columns

In [39]:
def distances_loop(X, Y):
    n, d = X.shape
    m = Y.shape[0]
    D = np.empty((n, m), dtype=np.float64)
    for i in range(n):
        for j in range(m):
            diff = X[i] - Y[j]           # (d,)
            D[i, j] = np.sqrt((diff*diff).sum())
    return D

# Beware: with n=4000, m=3000 this is extremely slow; use much smaller sizes if you try it.
# D_loop = distances_loop(X, Y)

### RIGHT: broadcasting with keepdims

In [40]:
# ||x - y||^2 = ||x||^2 + ||y||^2 - 2 x·y
X2 = (X*X).sum(axis=1, keepdims=True)        # (n,1)
Y2 = (Y*Y).sum(axis=1, keepdims=True).T      # (1,m)
XY = X @ Y.T                                 # (n,m)
D_sq = X2 + Y2 - 2.0*XY                      # (n,m), uses broadcasting for X2+Y2

# Numerical floor to zero (rounding may make tiny negatives)
np.maximum(D_sq, 0, out=D_sq)
D_matmul = np.sqrt(D_sq)


### semi-right: broadcasting with 3D temp

#### Why not just use `np.linalg.norm` for all-pairs distances?

You *can* write the pairwise Euclidean distances as:

```python
D = np.linalg.norm(X[:, None, :] - Y[None, :, :], axis=2)
```

✅ **Pros**
- Very clear and concise.  
- Uses broadcasting naturally: `(n, d)` → `(n, 1, d)` and `(1, m, d)` → `(n, m, d)`.

❌ **Cons**
- It first creates the full 3-D difference array `(n, m, d)` in memory.  
  For example, with `n=4000`, `m=3000`, `d=64`:

  ```
  4000 × 3000 × 64 × 8 bytes ≈ 6 GB
  ```

  That’s only the temporary array — easily enough to exhaust memory.


In [11]:
# D_try = np.linalg.norm(X[:, None, :] - Y[None, :, :], axis=2)